In [12]:
%load_ext autoreload
%autoreload 2

import datetime
import multiprocessing
import multiprocess
import feedparser as fp
import newspaper
from newspaper import news_pool
import helpers
from newspaper import Source, Article
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1.
This is downloading newspapers as one would conventionally using the $newspaper3k$ package 

In [13]:
%%time
# Do conventional newspaper3k build...
cnn_3k_paper = newspaper.build('https://www.cnn.com/', memoize_articles=False)
rt_3k_paper = newspaper.build('https://www.rt.com/', memoize_articles=False)
bbc_3k_paper = newspaper.build('https://www.bbc.com/news', memoize_articles=False)
fox_3k_paper = newspaper.build('https://www.foxnews.com/', memoize_articles=False)
msnbc_3k_paper = newspaper.build('https://www.msnbc.com/', memoize_articles=False)

CPU times: user 7.32 s, sys: 1.79 s, total: 9.11 s
Wall time: 17.7 s


In [14]:
%%time
# ... and multithread download:
papers = [cnn_3k_paper, 
          rt_3k_paper,
          bbc_3k_paper,
          fox_3k_paper,
          msnbc_3k_paper]
news_pool.set(papers, threads_per_source=2) # (5*2) = 10 threads total
news_pool.join()

CPU times: user 1min 13s, sys: 13.6 s, total: 1min 26s
Wall time: 5min 33s


In [15]:
%%time
# Process the downloaded texts:
for paper in papers:
    paper.parse_articles()

CPU times: user 7min 13s, sys: 13.5 s, total: 7min 26s
Wall time: 7min 41s


In [16]:
filepath = 'data/conventional_data_pulls/'

In [17]:
%%time
for paper in papers:
    helpers.write_out_articles(newspaper_obj=paper,
                               filepath=filepath, 
                               writeout_type='conventional')

CPU times: user 26.6 s, sys: 5.05 s, total: 31.7 s
Wall time: 37.9 s


Check for files:

In [18]:
df = pd.read_csv('data/conventional_data_pulls/2019_2_6_bbc.csv')

In [19]:
df['text'][95]

'Image copyright Nissan\n\nNissan\'s change of heart over making the X-Trail in Sunderland set Brexit alarm bells ringing over the weekend.\n\nIt was further evidence, to some, that car investment in the UK is drying up as we head towards the EU exit - ripping up promises made by Margaret Thatcher to Japanese manufacturers that they should consider the UK as their natural manufacturing outpost within the EU.\n\nWhile it is wrong to think that Brexit was the main reason Nissan pulled the X-Trail from Sunderland, it is still right to think that Brexit is proving a powerful deterrent for investment in the UK car industry.\n\nSome of the technical issues it throws up could start affecting UK car exports as soon as next week.\n\nBut there are three other important factors affecting the UK car industry.\n\n1. Diesel sales continue to fall in Europe\n\nNissan is not the only company to have been caught off guard by the sudden and rapid slump in diesel sales across Europe.\n\nThe VW emissions 